## DLC

In `config.yaml` specify the body parts and the training fraction `0.8`. Label frames and saving will create a `.csv` file in the `labeled-data` folder containing the label coordinates. After labeling the extracted frames, `create training dataset` will combine the labeled datasets from all the videos and split them to create train and test datasets under `dlc-models` folder. The training data will be used to train the network, while the test data set will be used for evaluating the network. Notice that you may create multiple separate training datasets by changing the shuffle number. 

- Modify the training network to only do 20 iterations. You can modify the training parameters in the GUI directory or in the `dlc-models\iteration0\train\pose_cfg.yaml` before training the network. 

Evaluate trained network will compute performance metrics such as train and test errors (**evaluation metric 1**, in `CombinedEvaluationResults` file) by comparing manual labels and predicted labels. They will be saved in the `evaluation-results` folder. 

Finally, you can use the network to analyze new videos (or the same video in this case). The prediction coordinates and likelihoods will be saved as an `.csv` file under the same directory as the video file. If you select the `plot trajectories` option in the GUI, it will create a `plot-poses` folder in the directory of the video, and it will contain some trajectory plots such as coordinates of body parts vs. time, likelihoods vs time.

You can also create labeled videos (**evaluation metric 2**, under `videos\mouse-top-viewDLC_resnet50_dlc-mouse-topviewJul24shuffle1_20_labeled.mp4`), modify the skeleton configurations in the `config.yaml` file to control what the skeleton should look like. 

- The `pcutoff` attribute in config.yaml controls the confidence / likelihood cutoff level that DLC uses to filter out predictions before creating the video. 

## DLC TRAINING (in progress)

Either create project in Home Folder, or move to Home Folder before (or after?) generating training sets

## Format Sleap Predictions for DLC Visualiztaion (blocked)

ISSUE: Cannot extract the labeled images file from .slp file, but DLC csv needs those paths and image files. 

In [2]:
import h5py
import matplotlib.pyplot as plt

filename = "C:/Users/louise.xu/Downloads/mouse-labels.h5" # or your own exported filename

with h5py.File(filename, "r") as f:
    dset_names = list(f.keys())
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]

    print("===filename===")
print(filename)
print()

print("===HDF5 datasets===")
print(dset_names)
print()

print("===locations (tracks) data shape===")
print(locations.shape)
print()

print("===nodes===")
for i, name in enumerate(node_names):
    print(f"{i}: {name}")
print()

===filename===
C:/Users/louise.xu/Downloads/mouse-labels.h5

===HDF5 datasets===
['edge_inds', 'edge_names', 'instance_scores', 'labels_path', 'node_names', 'point_scores', 'provenance', 'track_names', 'track_occupancy', 'tracking_scores', 'tracks', 'video_ind', 'video_path']

===locations (tracks) data shape===
(2330, 2, 2, 1)

===nodes===
0: head
1: tailBase



Need to formate csv into dlc acceptable format as instructed [here](https://github.com/DeepLabCut/DeepLabCut/wiki/Using-labeled-data-in-DeepLabCut-that-was-annotated-elsewhere-(or-merge-across-labelers))

In [98]:
import numpy as np
import pandas as pd
locations = locations.reshape((2330, 4))

bodyparts = []
coords = []
scorer = []
scorer_name = "louise"
for name in node_names:
    bodyparts += [name]*2
    coords += ["x"]
    coords += ["y"]
    scorer += [scorer_name]*2

def convert_df (arr):
    return pd.DataFrame(np.array([arr]))

df = pd.DataFrame(locations)

df = pd.concat([convert_df(bodyparts), convert_df(coords), df])
df.columns = scorer
df
#np.savetxt("C:/Users/louise.xu/Downloads/sleap-mouse-predictions.csv", locations_csv, delimiter=",", header=header, comments="")

,louise,louise,louise,louise
0,head,head,tailBase,tailBase
0,x,y,x,y
0,76.977142,88.586418,140.419189,187.721466
1,76.211014,87.9543,140.592499,180.707001
2,75.789993,84.666412,143.833466,176.909729
...,...,...,...,...
2325,347.900238,380.163879,424.336731,443.984863
2326,349.19101,380.117371,428.229675,443.702759
2327,352.273102,380.147186,428.39679,443.756714
2328,356.354065,380.305115,432.69458,440.235352


In [3]:
import deeplabcut
help(deeplabcut.create_labeled_video)
#%cd C:\Users\louise.xu\projects\Deeplabcut\examples\openfield-Pranav-2018-10-30
#deeplabcut.create_labeled_video('config.yaml', ['./videos/m3v1mp4.mp4'])

Help on function create_labeled_video in module deeplabcut.utils.make_labeled_video:

create_labeled_video(config, videos, videotype='', shuffle=1, trainingsetindex=0, filtered=False, fastmode=True, save_frames=False, keypoints_only=False, Frames2plot=None, displayedbodyparts='all', displayedindividuals='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False, color_by='bodypart', modelprefix='', init_weights='', track_method='', superanimal_name='', pcutoff=0.6, skeleton=[], skeleton_color='white', dotsize=8, colormap='rainbow', alphavalue=0.5, overwrite=False)
    Labels the bodyparts in a video.
    
    Make sure the video is already analyzed by the function
    ``deeplabcut.analyze_videos``.
    
    Parameters
    ----------
    config : string
        Full path of the config.yaml file.
    
    videos : list[str]
        A list of strings containing the full paths to videos for analysis or a path
        to the director